In [21]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import brown, stopwords
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer


In [22]:
brown.fileids()

['ca01',
 'ca02',
 'ca03',
 'ca04',
 'ca05',
 'ca06',
 'ca07',
 'ca08',
 'ca09',
 'ca10',
 'ca11',
 'ca12',
 'ca13',
 'ca14',
 'ca15',
 'ca16',
 'ca17',
 'ca18',
 'ca19',
 'ca20',
 'ca21',
 'ca22',
 'ca23',
 'ca24',
 'ca25',
 'ca26',
 'ca27',
 'ca28',
 'ca29',
 'ca30',
 'ca31',
 'ca32',
 'ca33',
 'ca34',
 'ca35',
 'ca36',
 'ca37',
 'ca38',
 'ca39',
 'ca40',
 'ca41',
 'ca42',
 'ca43',
 'ca44',
 'cb01',
 'cb02',
 'cb03',
 'cb04',
 'cb05',
 'cb06',
 'cb07',
 'cb08',
 'cb09',
 'cb10',
 'cb11',
 'cb12',
 'cb13',
 'cb14',
 'cb15',
 'cb16',
 'cb17',
 'cb18',
 'cb19',
 'cb20',
 'cb21',
 'cb22',
 'cb23',
 'cb24',
 'cb25',
 'cb26',
 'cb27',
 'cc01',
 'cc02',
 'cc03',
 'cc04',
 'cc05',
 'cc06',
 'cc07',
 'cc08',
 'cc09',
 'cc10',
 'cc11',
 'cc12',
 'cc13',
 'cc14',
 'cc15',
 'cc16',
 'cc17',
 'cd01',
 'cd02',
 'cd03',
 'cd04',
 'cd05',
 'cd06',
 'cd07',
 'cd08',
 'cd09',
 'cd10',
 'cd11',
 'cd12',
 'cd13',
 'cd14',
 'cd15',
 'cd16',
 'cd17',
 'ce01',
 'ce02',
 'ce03',
 'ce04',
 'ce05',
 'ce06',
 

In [23]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("/.*? ", " ",text)
    text = re.sub("\\./\\.", "",text)
    text = re.sub("``", "",text)
    text = re.sub("''", "",text)
    text = re.sub("  ", " ",text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
ca01 = brown.raw('ca01')
cc01 = brown.raw('cc01')
    
ca01 = text_cleaner(ca01)
cc01 = text_cleaner(cc01)

In [24]:
# Parse using SpaCy
nlp = spacy.load('en_core_web_sm')
ca01_doc = nlp(ca01)
cc01_doc = nlp(cc01)

In [25]:
# Group into sentences
ca01_sents = [[sent, 'ca01'] for sent in ca01_doc.sents]
cc01_sents = [[sent, 'cc01'] for sent in cc01_doc.sents]
# print(ca01_sents[:10])

# Combine
sentences = pd.DataFrame(ca01_sents + cc01_sents)
sentences

,0,1
0,"(The, Fulton, County, Grand, Jury, said, Frida...",ca01
1,"(The, jury, further, said, in, term, -, end, p...",ca01
2,"(Only, a, relative, handful, of, such, reports...",ca01
3,"(The, jury, said, it, did, find, that, many, o...",ca01
4,"(It, urged, that, the, city, take, steps, to, ...",ca01
5,"(It, urged, that, the, next, Legislature, prov...",ca01
6,"(This, is, one, of, the, major, items, in, the...",ca01
7,"(The, jurors, said, they, realize, a, proporti...",ca01
8,"(Nevertheless, ,, we, feel, that, in, the, fut...",ca01
9,"(The, jury, also, commented, on, the, Fulton, ...",ca01


In [26]:
# Excerpts

print(ca01_doc[:100])
print('\nca01 length:', len(ca01_doc))

print('\n', cc01_doc[:100])
print('\ncc01 length:', len(cc01_doc))

The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced no evidence that any irregularities took place The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , deserves the praise and thanks of the City of Atlanta for the manner in which the election was conducted The September-October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible irregularities in the hard-fought primary which was won by Mayor

ca01 length: 2133

 It is not news that Nathan Milstein is a wizard of the violin Certainly not in Orchestra Hall where he has played countless recitals , and where Thursday night he celebrated his 20th season with the Chicago Symphony Orchestra , playing the Brahms Concerto with his own slashing , demon-ridden cadenza melting into the high , pale , pure and lovely song with which a violinist unlocks the heart of the mus

In [27]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
ca01_words = bag_of_words(ca01_doc)
cc01_words = bag_of_words(cc01_doc)

# Combine bags to create a set of unique words.
common_words = set(ca01_words + cc01_words)

In [28]:
bow_data = bow_features(sentences, common_words)
bow_data.head()

Processing row 0


,summer,aj,promise,shortly,rally,87,wind,member,series,nathan,...,av,fully,only,also,liquor,score,straight,hot,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(The, Fulton, County, Grand, Jury, said, Frida...",ca01
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(The, jury, further, said, in, term, -, end, p...",ca01
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,"(Only, a, relative, handful, of, such, reports...",ca01
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(The, jury, said, it, did, find, that, many, o...",ca01
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(It, urged, that, the, city, take, steps, to, ...",ca01


# TF-IDF

In [29]:
print(sentences[0][0])

vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )

# Applying the vectorizer
X_tf = vectorizer.fit_transform(sentences[0].astype(str))
Y_tf = sentences[1]

X_train, X_test, y_train, y_test= train_test_split(X_tf, 
                                                    Y_tf,
                                                    test_size=0.4,
                                                    random_state=0)

# Reshapes the vectorizer output into something people can read
X_train_csr = X_train.tocsr()

#number of paragraphs
n = X_train_csr.shape[0]
#A list of dictionaries, one per paragraph
tfidf_bysent = [{} for _ in range(0,n)]
#List of features
terms = vectorizer.get_feature_names()


for i, j in zip(*X_train_csr.nonzero()):
    tfidf_bysent[i][terms[j]] = X_train_csr[i, j]

#Keep in mind that the log base 2 of 1 is 0, so a tf-idf score of 0 indicates that the word was present once in that sentence.
print('Tf_idf vector:', tfidf_bysent[0])

The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced no evidence that any irregularities took place
Tf_idf vector: {}


## BoW Vs. TF-IDF

## BoW with Random Forest

In [30]:
rfc = ensemble.RandomForestClassifier()
Y_bow = bow_data['text_source']
X_bow = np.array(bow_data.drop(['text_sentence','text_source'], 1))

X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bow, 
                                                    Y_bow,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train_bow, y_train_bow)

print('Training set score:', rfc.score(X_train_bow, y_train_bow))
print('\nTest set score:', rfc.score(X_test_bow, y_test_bow))

Training set score: 0.9824561403508771

Test set score: 0.8205128205128205


## TF-IDF with Random Forest

In [31]:
train = rfc.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))
print('cross val:', cross_val_score(rfc, X_train, y_train, cv=5))
pd.crosstab(y_train, rfc.predict(X_train))

(57, 323) (57,)
Training set score: 0.9824561403508771

Test set score: 0.7948717948717948
cross val: [0.75       0.75       0.63636364 0.81818182 0.81818182]


col_0,ca01,cc01
1,,
ca01,24,1
cc01,0,32


## BoW with Logistic Regression

In [32]:
lr = LogisticRegression()
train = lr.fit(X_train_bow, y_train_bow)
print(X_train_bow.shape, y_train_bow.shape)
print('Training set score:', lr.score(X_train_bow, y_train_bow))
print('\nTest set score:', lr.score(X_test_bow, y_test_bow))
print('cross val:', cross_val_score(lr, X_train_bow, y_train_bow, cv=5))
pd.crosstab(y_train_bow, lr.predict(X_train_bow))

(57, 1324) (57,)
Training set score: 1.0

Test set score: 0.9487179487179487
cross val: [0.91666667 0.91666667 0.90909091 0.81818182 0.90909091]


col_0,ca01,cc01
text_source,,
ca01,25,0
cc01,0,32


## TF-IDF with Logistic Regression

In [33]:
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
print('cross val:', cross_val_score(lr, X_train, y_train, cv=5))
pd.crosstab(y_train, lr.predict(X_train))

(57, 323) (57,)
Training set score: 1.0

Test set score: 0.8974358974358975
cross val: [0.91666667 0.91666667 0.72727273 0.72727273 0.72727273]


col_0,ca01,cc01
1,,
ca01,25,0
cc01,0,32


# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [34]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train_bow, y_train_bow)

print('Training set score:', clf.score(X_train_bow, y_train_bow))
print('\nTest set score:', clf.score(X_test_bow, y_test_bow))
print('cross val:', cross_val_score(clf, X_train_bow, y_train_bow, cv=5))
pd.crosstab(y_train_bow, clf.predict(X_train_bow))

Training set score: 1.0

Test set score: 0.8717948717948718
cross val: [0.83333333 0.83333333 0.63636364 0.72727273 0.63636364]


col_0,ca01,cc01
text_source,,
ca01,25,0
cc01,0,32


## TF-IDF with Gradient Boosting

In [35]:
train = clf.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))
print('cross val:', cross_val_score(clf, X_train, y_train, cv=5))
pd.crosstab(y_train, clf.predict(X_train))

(57, 323) (57,)
Training set score: 1.0

Test set score: 0.8717948717948718
cross val: [0.83333333 0.75       0.54545455 0.72727273 0.81818182]


col_0,ca01,cc01
1,,
ca01,25,0
cc01,0,32


# BoW with SVC

In [36]:
svc = SVC()
train = svc.fit(X_train_bow, y_train_bow)
print(X_train_bow.shape, y_train_bow.shape)
print('Training set score:', svc.score(X_train_bow, y_train_bow))
print('\nTest set score:', svc.score(X_test_bow, y_test_bow))

(57, 1324) (57,)
Training set score: 0.5614035087719298

Test set score: 0.6410256410256411


## TF-IDF with SVC

In [37]:
train = svc.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))
print('cross val:', cross_val_score(svc, X_train, y_train, cv=5))
pd.crosstab(y_train, svc.predict(X_train))

(57, 323) (57,)
Training set score: 0.5614035087719298

Test set score: 0.6410256410256411
cross val: [0.58333333 0.58333333 0.54545455 0.54545455 0.54545455]


col_0,cc01
1,
ca01,25
cc01,32


 # Pick model and improve by 5%


## BoW Random Forest

In [38]:
rfc = ensemble.RandomForestClassifier()
Y_bow = bow_data['text_source']
X_bow = np.array(bow_data.drop(['text_sentence','text_source'], 1))

X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bow, 
                                                    Y_bow,
                                                    test_size=0.1,
                                                    random_state=0)
train = rfc.fit(X_train_bow, y_train_bow)

print('Training set score:', rfc.score(X_train_bow, y_train_bow))
print('\nTest set score:', rfc.score(X_test_bow, y_test_bow))

Training set score: 1.0

Test set score: 1.0


In [39]:
# Changed test size from 0.4 to 0.1 to increase accuracy